In [1]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_parquet("../data/raw/puc_vendas.parquet")
print(f"Number of sales: {df.shape[0]}")
df.head()

Number of sales: 9271036


,COD_CUPOM,COD_CLIENTE,CLIENTE_FISICO_JURIDICO,SEXO_CLIENTE,DTNASCIMENTO_CLIENTE,COD_SKU,SKU,CATEGORIA_SKU,SUBCATEGORIA_SKU,COD_LOJA,UF_CIDADE,COD_CIDADE,NOME_CIDADE,DATA_CUPOM,UNIDADES,IDENTIFICADOR_PROMOCIONAL,PRECO_REGULAR,TOTAL_DESCONTO,TOTAL_BRUTO,TOTAL_LIQUIDO
0,782344,55750557.0,Pessoa Física,F,1980-11-18,10085918,VONAU FLASH 4MG 10CP,MEDICAMENTOS,REFERENCIA,764,RS,80400,PORTO ALEGRE,2024-04-11 17:13:25,1.0,NaN,47.03,12.70,47.03,34.33
1,759507,55750557.0,Pessoa Física,F,1980-11-18,10106687,APTANUTRI PREMIUM 3,NUTRICAO,NUTRICAO LEITES,764,RS,80400,PORTO ALEGRE,2024-02-19 15:47:17,1.0,NaN,73.99,14.80,73.99,59.19
2,742082,55750557.0,Pessoa Física,F,1980-11-18,10037705,TORRADA ISABELA 142G TRADICIONAL,CONVENIENCIA,CONVENIENCIA PERECIVEIS,764,RS,80400,PORTO ALEGRE,2024-01-05 19:48:52,1.0,NaN,5.99,0.00,5.99,5.99
3,747733,55750557.0,Pessoa Física,F,1980-11-18,10004419,OZEMPIC 1MG 4AGULHAS NOVOFINE,MEDICAMENTOS,PBM ESPECIAL,764,RS,80400,PORTO ALEGRE,2024-01-20 14:41:56,1.0,NaN,1220.52,254.37,1220.52,966.15
4,748216,55750557.0,Pessoa Física,F,1980-11-18,10030435,NOVALGINA 1G 20CP OPELLA,MEDICAMENTOS,REFERENCIA ONEROSOS,764,RS,80400,PORTO ALEGRE,2024-01-22 16:48:36,1.0,100271131.0,41.51,5.52,41.51,35.99


In [3]:
customers = pd.read_parquet("../data/cleaned/customers.parquet")
products = pd.read_parquet("../data/cleaned/products.parquet")
sales = df[(df.COD_CLIENTE.isin(customers.client_id)) & (df.COD_SKU.isin(products.product_id))]
print(f"New number of sales: {sales.shape[0]}")

New number of sales: 6899984


In [4]:
sales.head()

,COD_CUPOM,COD_CLIENTE,CLIENTE_FISICO_JURIDICO,SEXO_CLIENTE,DTNASCIMENTO_CLIENTE,COD_SKU,SKU,CATEGORIA_SKU,SUBCATEGORIA_SKU,COD_LOJA,UF_CIDADE,COD_CIDADE,NOME_CIDADE,DATA_CUPOM,UNIDADES,IDENTIFICADOR_PROMOCIONAL,PRECO_REGULAR,TOTAL_DESCONTO,TOTAL_BRUTO,TOTAL_LIQUIDO
1,759507,55750557.0,Pessoa Física,F,1980-11-18,10106687,APTANUTRI PREMIUM 3,NUTRICAO,NUTRICAO LEITES,764,RS,80400,PORTO ALEGRE,2024-02-19 15:47:17,1.0,NaN,73.99,14.80,73.99,59.19
2,742082,55750557.0,Pessoa Física,F,1980-11-18,10037705,TORRADA ISABELA 142G TRADICIONAL,CONVENIENCIA,CONVENIENCIA PERECIVEIS,764,RS,80400,PORTO ALEGRE,2024-01-05 19:48:52,1.0,NaN,5.99,0.00,5.99,5.99
3,747733,55750557.0,Pessoa Física,F,1980-11-18,10004419,OZEMPIC 1MG 4AGULHAS NOVOFINE,MEDICAMENTOS,PBM ESPECIAL,764,RS,80400,PORTO ALEGRE,2024-01-20 14:41:56,1.0,NaN,1220.52,254.37,1220.52,966.15
4,748216,55750557.0,Pessoa Física,F,1980-11-18,10030435,NOVALGINA 1G 20CP OPELLA,MEDICAMENTOS,REFERENCIA ONEROSOS,764,RS,80400,PORTO ALEGRE,2024-01-22 16:48:36,1.0,100271131.0,41.51,5.52,41.51,35.99
5,762048,55750557.0,Pessoa Física,F,1980-11-18,10027971,SALGADINHO ELMA CHIPS PINGO DOURO BACON,CONVENIENCIA,CONVENIENCIA PERECIVEIS,764,RS,80400,PORTO ALEGRE,2024-02-24 19:30:35,1.0,NaN,7.99,0.00,7.99,7.99


In [5]:
sales.COD_CLIENTE.nunique(), customers.client_id.nunique()

(739233, 739233)

In [6]:
sales.COD_SKU.nunique(), products.product_id.nunique()

(18009, 18009)

In [7]:
# remove from customers all rows where client_id is not int sales
customers = customers[customers.client_id.isin(sales.COD_CLIENTE)]
customers.shape[0]

739233

In [8]:
# remove from products all rows where product_id is not in sales
products = products[products.product_id.isin(sales.COD_SKU)]
products.shape[0]

18009

In [9]:
sales = sales[[
    "COD_CUPOM", 
    "COD_CLIENTE", 
    "COD_SKU", 
    "DATA_CUPOM",
    "COD_LOJA", 
    "UNIDADES", 
    "IDENTIFICADOR_PROMOCIONAL", 
    "PRECO_REGULAR", 
    "TOTAL_DESCONTO", 
    "TOTAL_BRUTO", 
    "TOTAL_LIQUIDO"
]]
sales.head()

,COD_CUPOM,COD_CLIENTE,COD_SKU,DATA_CUPOM,COD_LOJA,UNIDADES,IDENTIFICADOR_PROMOCIONAL,PRECO_REGULAR,TOTAL_DESCONTO,TOTAL_BRUTO,TOTAL_LIQUIDO
1,759507,55750557.0,10106687,2024-02-19 15:47:17,764,1.0,NaN,73.99,14.80,73.99,59.19
2,742082,55750557.0,10037705,2024-01-05 19:48:52,764,1.0,NaN,5.99,0.00,5.99,5.99
3,747733,55750557.0,10004419,2024-01-20 14:41:56,764,1.0,NaN,1220.52,254.37,1220.52,966.15
4,748216,55750557.0,10030435,2024-01-22 16:48:36,764,1.0,100271131.0,41.51,5.52,41.51,35.99
5,762048,55750557.0,10027971,2024-02-24 19:30:35,764,1.0,NaN,7.99,0.00,7.99,7.99


In [10]:
sales["sale"] = sales[['COD_CLIENTE', 'COD_LOJA', 'DATA_CUPOM']].astype(str).agg('_'.join, axis=1)
sales["sale_id"] = sales.groupby('sale').ngroup() + 1
sales.drop(columns=["sale"], inplace=True)
sales.head()

,COD_CUPOM,COD_CLIENTE,COD_SKU,DATA_CUPOM,COD_LOJA,UNIDADES,IDENTIFICADOR_PROMOCIONAL,PRECO_REGULAR,TOTAL_DESCONTO,TOTAL_BRUTO,TOTAL_LIQUIDO,sale_id
1,759507,55750557.0,10106687,2024-02-19 15:47:17,764,1.0,NaN,73.99,14.80,73.99,59.19,2270794
2,742082,55750557.0,10037705,2024-01-05 19:48:52,764,1.0,NaN,5.99,0.00,5.99,5.99,2270742
3,747733,55750557.0,10004419,2024-01-20 14:41:56,764,1.0,NaN,1220.52,254.37,1220.52,966.15,2270750
4,748216,55750557.0,10030435,2024-01-22 16:48:36,764,1.0,100271131.0,41.51,5.52,41.51,35.99,2270752
5,762048,55750557.0,10027971,2024-02-24 19:30:35,764,1.0,NaN,7.99,0.00,7.99,7.99,2270806


In [11]:
sales.rename(
    columns={
        "COD_CUPOM": "cod_cupon",
        "COD_CLIENTE": "client_id",
        "COD_SKU": "product_id",
        "COD_LOJA": "store_id",
        "DATA_CUPOM": "date",
        "UNIDADES": "units",
        "IDENTIFICADOR_PROMOCIONAL": "promotion_id",
        "PRECO_REGULAR": "regular_price",
        "TOTAL_DESCONTO": "discount",
        "TOTAL_BRUTO": "gross_total",
        "TOTAL_LIQUIDO": "net_total",
    }, inplace=True
)

In [12]:
sales["week_of_year"] = sales.date.dt.isocalendar().week
sales["day_of_week"] = sales.date.dt.dayofweek
sales["hour"] = sales.date.dt.hour
sales["minute"] = sales.date.dt.minute

In [13]:
sales["was_in_promotion"] = sales.promotion_id.notnull().astype(int)
sales.drop(columns=["promotion_id"], inplace=True)

In [14]:
sales = sales[[
    "cod_cupon",
    "sale_id",
    "client_id",
    "product_id",
    "store_id",
    "week_of_year",
    "day_of_week",
    "hour",
    "minute",
    "units",
    "was_in_promotion",
    "regular_price",
    "discount",
    "gross_total",
    "net_total",
]]

In [15]:
sales.head(30)

,cod_cupon,sale_id,client_id,product_id,store_id,week_of_year,day_of_week,hour,minute,units,was_in_promotion,regular_price,discount,gross_total,net_total
1,759507,2270794,55750557.0,10106687,764,8,0,15,47,1.0,0,73.99,14.80,73.99,59.19
2,742082,2270742,55750557.0,10037705,764,1,4,19,48,1.0,0,5.99,0.00,5.99,5.99
3,747733,2270750,55750557.0,10004419,764,3,5,14,41,1.0,0,1220.52,254.37,1220.52,966.15
4,748216,2270752,55750557.0,10030435,764,4,0,16,48,1.0,1,41.51,5.52,41.51,35.99
5,762048,2270806,55750557.0,10027971,764,8,5,19,30,1.0,0,7.99,0.00,7.99,7.99
7,769369,2270826,55750557.0,2640,764,11,2,12,41,2.0,0,20.74,10.37,41.48,31.11
8,781620,2270841,55750557.0,100010880,764,15,2,12,27,3.0,0,315.26,350.91,945.78,594.87
9,756251,2270778,55750557.0,1021,764,6,4,17,48,1.0,0,18.31,3.66,18.31,14.65
10,776686,2270835,55750557.0,10039091,764,13,5,13,28,1.0,1,41.90,4.00,41.90,37.90
11,761524,2270802,55750557.0,999786,764,8,4,15,58,1.0,0,6.90,0.00,6.90,6.90


In [16]:
sales.to_parquet("../data/cleaned/sales.parquet")
customers.to_parquet("../data/cleaned/customers.parquet")
products.to_parquet("../data/cleaned/products.parquet")